In [10]:
# модуль библиотеки OpenCV для работы с изображениями
import cv2

# всякие полезные библиотеки для работы с матрицами / отрисовки / etc.
import glob
import tqdm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

plt.rcParams['figure.figsize'] = (12, 5)
sns.set_style("white")

import os
from os.path import basename # чтобы достать имя файла из пути

from imgaug import augmenters as iaa

Самолёты я беру из следующих аэропортов:

vko18-gr-small.jpg

jto17-gr1-small.jpg

dme18-gr1-small.jpg.

Т.к. будет аугментация (поворот), то надо брать изображение, где самолёт далеко от краёв, чтобы оно влезло

# Подбираем параметры порогового фильтра

Берём серые картинки и смотрим что будет при разных параметрах фильтра.

In [ ]:
PATH_THR = "/srv/samba/share/data/airbus/train/12/"
families_list = glob.glob(PATH_THR + "*.*") # получаем список имен файлов с изображениями
print(families_list)

Разбираюсь, как работает thresholding in OpenCV http://www.learnopencv.com/opencv-threshold-python-cpp/, и http://robocraft.ru/blog/computervision/357.html

In [ ]:
PATH_TEST_IMG = '/srv/samba/share/data/airbus/train/12/1.jpg'
img = cv2.imread(PATH_TEST_IMG,cv2.IMREAD_GRAYSCALE)

In [ ]:
def threshold_test(img, thr, maxv, filename='none'):
    ret,thresh = cv2.threshold(img,thr,maxv,cv2.THRESH_BINARY_INV)
    titles = ['Original Image '+filename,'BINARY_INV', 'BINARY']
    images = [img, thresh]
    for i in range(2):
        plt.subplot(1,3,i+1), plt.imshow(images[i])
        plt.title(titles[i])
        plt.xticks([]), plt.yticks([])
    plt.subplot(1,3,3),plt.hist(images[0].ravel(),256)
    plt.title('Hist')
    plt.show()
#     return thresh

In [ ]:
plt.rcParams['figure.figsize'] = (10, 2)
thr = 190
maxv = 250
threshold_test(img, 190, 250, basename(PATH_TEST_IMG))

In [ ]:
j = 0
for family in families_list:
    j += 1
    if j<20:
        continue
    if j>30:
        break
    print('j='+str(j)+' ------------------------------------------------------------------------------------')
    img = cv2.imread(family,cv2.IMREAD_GRAYSCALE)
    for thr in range(200,250,25):
        print('thr='+str(thr)+', maxv='+str(maxv))
        
        threshold_test(img, thr, maxv, basename(family))
    

Видим, что для большинства самолётов подходят параметры threshold = 175, 200, 235.

Остались ли изображения ещё - не знаю. На всякий случй сделал двойное фильтрование с порогом сверху.

In [ ]:
PATH_TEST_IMG = '/srv/samba/share/data/airbus/train/12/1.jpg'
img = cv2.imread(PATH_TEST_IMG,cv2.IMREAD_GRAYSCALE)

In [ ]:
def threshold_test_diap(img, thr_min, thr_max, maxv, filename='none'):
    ret,thresh = cv2.threshold(img,thr_max,maxv,cv2.THRESH_TRUNC)
    ret,thresh = cv2.threshold(thresh,thr_min,maxv,cv2.THRESH_BINARY)
    titles = ['Original Image '+filename,'BINARY_INV', 'BINARY']
    images = [img, thresh]
    for i in range(2):
        plt.subplot(1,3,i+1), plt.imshow(images[i])
        plt.title(titles[i])
        plt.xticks([]), plt.yticks([])
    plt.subplot(1,3,3),plt.hist(images[0].ravel(),256)
    plt.title('Hist')
    plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = (10, 2)
thr_min = 100
thr_max = 200
threshold_test_diap(img, thr_min, thr_max, 255, basename(PATH_TEST_IMG))

In [ ]:
j = 0
for family in families_list:
    j += 1
#     if j in [8,9,10, 11,12,19]:
    if j in [8,9,10, 11,12,19]:
        print('j='+str(j)+' ------------------------------------------------------------------------------------')
        img = cv2.imread(family,cv2.IMREAD_GRAYSCALE)
        for thr in range(225,251,10):
            print('thr='+str(thr)+', maxv='+str(maxv))
#             threshold_test_diap(img, 100, thr, maxv, basename(family))
            threshold_test(img, thr, maxv, basename(family))
    

### Проверяем эти фильтры на трейновых изображениях



In [ ]:
PATH_TESTTHR = "/srv/samba/share/data/airbus/test/color/"
families_list = glob.glob(PATH_TESTTHR + "*.*") # получаем список имен файлов с изображениями

In [ ]:
plt.rcParams['figure.figsize'] = (15, 2)
maxv = 255
j = 0
for family in families_list:
    j += 1
#     if j<0:
#         continue
    if j>1:
        break
    print('j='+str(j)+' ------------------------------------------------------------------------------------')
    img = cv2.imread(family,cv2.IMREAD_GRAYSCALE)
    for thr in [175, 200, 235]:
        print('thr='+str(thr)+', maxv='+str(maxv))        
        threshold_test(img, thr, maxv, basename(family))

Проверим на большом изображении

In [ ]:
PATH_TESTTHR = "/srv/samba/share/data/airbus/test/color/1/"
families_list = glob.glob(PATH_TESTTHR + "*.*") # получаем список имен файлов с изображениями
print(families_list)
family = families_list[0]
img = cv2.imread(family,cv2.IMREAD_GRAYSCALE)
del family

In [ ]:
plt.rcParams['figure.figsize'] = (30, 30)
thr=175
maxv = 255

ret,thresh = cv2.threshold(img,thr,maxv,cv2.THRESH_BINARY)
plt.imshow(thresh)
plt.xticks([]), plt.yticks([])
plt.show()

### Логика работы

Теперь понятна логика работы алгоритма. 
1. Загружаем исходное изображение и применяем 3 раза пороговую фильтацию с разными значениями порога. Сохраняем эти изображения
2. На полученных изображениях проверяем наш ХОГ
3. Везде, где превышен ПОРОГ алгоритма мы отмечаем окно
4. Сохраняем окна

### Теперь разберёмся с аугментацией

In [ ]:
seq = iaa.Sequential([
    iaa.Affine(rotate=(45) )
])

# seq = iaa.Sequential([
#     iaa.SomeOf((0, 5), iaa.Affine(rotate=(-45,45)) )
# ])


In [ ]:
img = cv2.imread('/srv/samba/share/data/airbus/train/test-aug.jpg',cv2.IMREAD_GRAYSCALE)
# print(img)
# print(img.size)
# print(img.shape)
imgList = np.asarray(img).reshape(128,128,1) # Grayscale images must have shape (height, width, 1) each.
# imgList = [imgList, imgList, imgList]
# print(imgList)
# print(imgList.size)
# print(imgList.shape)
img_aug = seq.augment_image(imgList)
# print(img_aug)
# print(img_aug.shape)
img_aug = np.asarray(img_aug).reshape(128,128)
# print(img_aug.shape)

In [ ]:
np.asarray(img_aug[3]).reshape(128,128)

In [ ]:
plt.subplot(1,2,1), plt.imshow(img)
plt.subplot(1,2,2), plt.imshow(img_aug)
plt.show()

In [ ]:
seq.show_grid(img, cols=8, rows=8)

# 1. Подгружаем данные, смотрим на них

In [52]:
pos_size = 384
neg_size = 539 # Будет лучше, если положительных и отрицательных примеров будет поровну
PATH_POS = "/srv/samba/share/data/airbus/train/16/"
# 11 = 57
# 12 = 30 <---
# 13 = 97
# 14 = 101
# 15 = 720
# 16 = 384

PATH_NEG = "/srv/samba/share/data/airbus/fail/14/"
# 11 = 40 <---
# 12 = 960
# 13 = 539
# 14 = 4292

### обучающая выборка, самолёты

 По идее ОБРЕЗАНИЕ (image = cv2.resize(cv2.imread(name, cv2.IMREAD_GRAYSCALE), (64, 64))) не надо делать, потому что потом надо делать аугментацию. Лучше это сделать после аугментации

In [39]:
def read_faces(path, number):
    """
    Функция для загрузки изображений из папки по пути path.
    """
    names = glob.glob(path + "*.*")
    data = []
    for name in tqdm.tqdm_notebook(names[:number]):
#         image = cv2.resize(cv2.imread(name, cv2.IMREAD_GRAYSCALE), (64, 64))
        image = cv2.imread(name, cv2.IMREAD_GRAYSCALE)
        data.append(image)

    return np.array(data)

In [ ]:
# с аугментацией
def read_faces(path, number):
    """
    Функция для загрузки изображений из папки по пути path.
    """
    names = glob.glob(path + "*.*")
    data = []
    maxv = 255
    
    for name in tqdm.tqdm_notebook(names[:number]):
#         image = cv2.resize(cv2.imread(name, cv2.IMREAD_GRAYSCALE), (64, 64))
        image = cv2.imread(name, cv2.IMREAD_GRAYSCALE)
#         data.append(cv2.pyrUp(image[25:90, 32:100])) # обрезаем картинку.
#         data.append(image)

        # применяем фильтр
#         x1,x2, y1,y2 = 0,128, 0,128
        x1,x2, y1,y2 = 20,100, 20,100
        size = (64, 64)
        angle_from, angle_to, angle_step = 0, 360, 45
        
        image6464 = np.asarray(image).reshape(128,128,1) # (height, width, 1)
        for angle in range(angle_from,angle_to, angle_step):
            seq = iaa.Sequential([ iaa.Affine(rotate=angle) ])
            img_aug = seq.augment_image(image6464)
            img_aug = np.asarray(img_aug).reshape(128,128)

            ret,thresh = cv2.threshold(img_aug[x1:x2, y1:y2],175,maxv,cv2.THRESH_BINARY_INV)
            data.append(cv2.resize(thresh, size))
            ret,thresh = cv2.threshold(img_aug[x1:x2, y1:y2],200,maxv,cv2.THRESH_BINARY_INV)
            data.append(cv2.resize(thresh, size))
            ret,thresh = cv2.threshold(img_aug[x1:x2, y1:y2],235,maxv,cv2.THRESH_BINARY_INV)
            data.append(cv2.resize(thresh, size))
            
#         plt.rcParams['figure.figsize'] = (20, 20)
#         for i in range(1,len(data)):
#             plt.subplot(5,5,i), plt.imshow(data[i])
#         plt.subplot(5,5,len(data)), plt.imshow(image)
#         plt.show

    return np.array(data)

In [40]:
data_pos = read_faces(path=PATH_POS, number=pos_size) # получим np.array с N_FACES изображениями лиц размера (64, 64)
labels_pos = np.ones(shape=(len(data_pos), 1)) # np.array c единичками 
print("Read %d images, \ndata_pos shape is %s, \nlabels_pos shape is %s" % (pos_size, data_pos.shape, labels_pos.shape))


Read 384 images, 
data_pos shape is (384, 64, 64), 
labels_pos shape is (384, 1)


In [ ]:
def show_random_images(data, size = 5):
    """
    Функция для отрисовки size x size случайных изображений из data.
    """
    random_idxs = np.random.randint(0, len(data), size ** 2)
    
    plt.figure(figsize=(12 ,5))
    for i in range(size):
        for j in range(size):
            
            num = i * size + j
            plt.subplot(size, size, num + 1)
            pic = cv2.resize(data[num], (64, 64))
            pic = data[num]
            plt.imshow(pic, cmap='gray')
            plt.axis('off')
    
    plt.show()

In [ ]:
# print(data_pos[0])
# print( cv2.resize(data_pos[0],(64,64)) )

plt.subplot(1,2,1), plt.imshow(data_pos[0])
# plt.subplot(1,2,2), plt.imshow(cv2.resize(data_pos[0][20:100,20:100] ,(64,64)))
# plt.subplot(1,2,2), plt.imshow( data_pos[0][20:100,20:100] )
plt.show
# print( len(data_pos[0][20:84,20:84]) )

In [ ]:
show_random_images(data_pos, size=4)

In [ ]:
# сохраняем выборку на диск
i = 0
for item in data_pos:
    cv2.imwrite('faces/'+str(i)+'.jpg',item)
    i += 1

### обучающая выборка, фоны

In [41]:
def read_backgrounds(path, number):
    names = glob.glob(path + "*.*")
    data = []
    for name in tqdm.tqdm_notebook(names[:number]):
        image = cv2.resize(cv2.imread(name, cv2.IMREAD_GRAYSCALE), (64, 64))
#         image = cv2.imread(name, cv2.IMREAD_GRAYSCALE)
        data.append(image)
    return np.array(data)

In [ ]:
def read_backgrounds(path, number):
    names = glob.glob(path + "*.*")
    data = []
    maxv = 255
    for name in tqdm.tqdm_notebook(names[:number]):
#         image = cv2.resize(cv2.imread(name, cv2.IMREAD_GRAYSCALE), (64, 64))
        image = cv2.imread(name, cv2.IMREAD_GRAYSCALE)
#         data.append(cv2.pyrUp(image[25:90, 32:100])) # обрезаем картинку.
#         data.append(image)

        # применяем фильтр
#         x1,x2, y1,y2 = 0,128, 0,128
        x1,x2, y1,y2 = 20,100, 20,100
        size = (64, 64)
        angle_from, angle_to, angle_step = 0, 360, 45
        
        image6464 = np.asarray(image).reshape(128,128,1) # (height, width, 1)
        for angle in range(angle_from,angle_to, angle_step):
            seq = iaa.Sequential([ iaa.Affine(rotate=angle) ])
            img_aug = seq.augment_image(image6464)
            img_aug = np.asarray(img_aug).reshape(128,128)

            ret,thresh = cv2.threshold(img_aug[x1:x2, y1:y2],175,maxv,cv2.THRESH_BINARY_INV)
            data.append(cv2.resize(thresh, size))
            ret,thresh = cv2.threshold(img_aug[x1:x2, y1:y2],200,maxv,cv2.THRESH_BINARY_INV)
            data.append(cv2.resize(thresh, size))
            ret,thresh = cv2.threshold(img_aug[x1:x2, y1:y2],235,maxv,cv2.THRESH_BINARY_INV)
            data.append(cv2.resize(thresh, size))

#         plt.rcParams['figure.figsize'] = (5, 5)
#         plt.imshow(data[3])
#         plt.show
#         break
    return np.array(data)

In [42]:
data_neg = read_backgrounds(path=PATH_NEG, number=neg_size)
labels_neg = np.zeros((len(data_neg), 1)) # np.array с нулями
print("Read %d images, \ndata_neg shape is %s, \nlabels_neg shape is %s" % (neg_size, data_neg.shape, labels_neg.shape))

# for one iamge we make (angle_to-angle_from)/angle_step + "1 original"  images
# size of out must be  [(angle_to-angle_from)/angle_step + "1 original" ]* "count of images"
# so [(360-45)/45 + 1] * 40 = 960


Read 539 images, 
data_neg shape is (539, 64, 64), 
labels_neg shape is (539, 1)


In [ ]:
show_random_images(data_neg, size=4)

In [ ]:
# сохраняем выборку на диск
i = 0
for item in data_neg:
    cv2.imwrite('faces/'+str(i)+'.jpg',item)
    i += 1

#  Аугментация для произвольных изображений

In [19]:
def read_faces(path):
    """
    Функция для загрузки изображений из папки по пути path.
    """
    names = glob.glob(path + "*.*")
    data = []
    maxv = 255
    
    for name in tqdm.tqdm_notebook(names):
        image = cv2.resize(cv2.imread(name, cv2.IMREAD_GRAYSCALE), (64, 64))
#         image = cv2.imread(name, cv2.IMREAD_GRAYSCALE)

        # применяем фильтр
#         x1,x2, y1,y2 = 0,128, 0,128
        size = (64, 64)
        angle_from, angle_to, angle_step = 0, 360, 45
        
        i=0
        image6464 = np.asarray(image).reshape(64,64,1) # (height, width, 1)
        for angle in range(angle_from,angle_to, angle_step):
            seq = iaa.Sequential([ iaa.Affine(rotate=angle) ])
            img_aug = seq.augment_image(image6464)
            img_aug = np.asarray(img_aug).reshape(64,64)

            cv2.imwrite('faces_aug/'+name.split('/')[-1]+str(i)+'.jpg', (cv2.resize(img_aug, size)) )
            print('faces_aug/'+name.split('/')[-1]+str(i)+'.jpg')
            i+=1
            
#         plt.rcParams['figure.figsize'] = (20, 20)
#         for i in range(1,len(data)):
#             plt.subplot(5,5,i), plt.imshow(data[i])
#         plt.subplot(5,5,len(data)), plt.imshow(image)
#         plt.show

    return np.array(data)

In [20]:
read_faces("/srv/samba/share/data/airbus/fail/faces/")

Widget Javascript not detected.  It may not be installed or enabled properly.


faces_aug/10_2.jpg0.jpg
faces_aug/10_2.jpg1.jpg
faces_aug/10_2.jpg2.jpg
faces_aug/10_2.jpg3.jpg
faces_aug/10_2.jpg4.jpg
faces_aug/10_2.jpg5.jpg
faces_aug/10_2.jpg6.jpg
faces_aug/10_2.jpg7.jpg
faces_aug/12_1.jpg0.jpg
faces_aug/12_1.jpg1.jpg
faces_aug/12_1.jpg2.jpg
faces_aug/12_1.jpg3.jpg
faces_aug/12_1.jpg4.jpg
faces_aug/12_1.jpg5.jpg
faces_aug/12_1.jpg6.jpg
faces_aug/12_1.jpg7.jpg
faces_aug/1_1.jpg0.jpg
faces_aug/1_1.jpg1.jpg
faces_aug/1_1.jpg2.jpg
faces_aug/1_1.jpg3.jpg
faces_aug/1_1.jpg4.jpg
faces_aug/1_1.jpg5.jpg
faces_aug/1_1.jpg6.jpg
faces_aug/1_1.jpg7.jpg
faces_aug/4_2.jpg0.jpg
faces_aug/4_2.jpg1.jpg
faces_aug/4_2.jpg2.jpg
faces_aug/4_2.jpg3.jpg
faces_aug/4_2.jpg4.jpg
faces_aug/4_2.jpg5.jpg
faces_aug/4_2.jpg6.jpg
faces_aug/4_2.jpg7.jpg
faces_aug/2_1.jpg0.jpg
faces_aug/2_1.jpg1.jpg
faces_aug/2_1.jpg2.jpg
faces_aug/2_1.jpg3.jpg
faces_aug/2_1.jpg4.jpg
faces_aug/2_1.jpg5.jpg
faces_aug/2_1.jpg6.jpg
faces_aug/2_1.jpg7.jpg
faces_aug/12_2.jpg0.jpg
faces_aug/12_2.jpg1.jpg
faces_aug/12_2.j

array([], dtype=float64)